In [ ]:
from anomaly_qwen2_5_vl import Anomaly_Qwen2_5_VLForConditionalGeneration, Qwen2_5_VLProcessor
from qwen_vl_utils import process_vision_info

# # default: Load the model modelon the available device(s)
# model = Anomaly_Qwen2_5_VLForConditionalGeneration.from_pretrained(
#     "Qwen/Qwen2.5-VL-7B-Instruct", torch_dtype="auto", device_map="auto"
# )
#
# # default processor
# processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")

# The default range for the number of visual tokens per image in the model is 4-16384.
# You can set min_pixels and max_pixels according to your needs, such as a token range of 256-1280, to balance performance and cost.
# min_pixels = 256*28*28
# max_pixels = 1280*28*28
# processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels)

# save_dir = "Geo/Qwen2.5-VL-7B-Instruct"
# model.save_pretrained(save_dir)
# processor.save_pretrained(save_dir)

# default: Load the model on the available device(s)
model = Anomaly_Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Geo/Anomaly_Qwen2.5-VL-7B-Instruct", torch_dtype="auto", device_map="auto"
)

# default processor
processor = Qwen2_5_VLProcessor.from_pretrained("Geo/Anomaly_Qwen2.5-VL-7B-Instruct")

In [ ]:
# Messages containing a local video path and a text query
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "video",
                "video": "./dataset/train/01_0029-180_203.mp4",
                "fps": 24.0,
            },
            {
                "type": "text",
                "text": "这段校园步行道视频里有异常情况吗？"
            },
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs, video_kwargs = process_vision_info(messages, return_video_kwargs=True)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
    **video_kwargs,
)
inputs = inputs.to("cuda")

# Inference
generated_ids = model.generate(**inputs, max_new_tokens=512)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
# generated_ids_trimmed = [
#     out_ids for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
# ]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)
print(video_inputs[0].shape)

In [ ]:
inputs.keys()

In [ ]:
anomaly_maps = model.model.anomaly_maps
anomaly_maps.shape

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np

# 假设你的数据
data = torch.softmax(anomaly_maps, dim=1)
data = data.float().cpu().numpy()
print(data.min(), data.max())
# 强制数值到 0-1 区间
data = np.clip(data, 0, 1)

num_frames, num_channels, H, W = data.shape
ncols = 6
nrows = (num_frames + ncols - 1) // ncols

plt.figure(figsize=(ncols * 3, nrows * 3 * num_channels))

for i in range(num_frames):
    for c in range(num_channels):
        plt.subplot(nrows * num_channels, ncols, i + 1 + c * nrows * ncols)
        plt.imshow(data[i, c], vmin=0, vmax=1)  # 再加保险
        plt.axis('off')
        plt.title(f'Frame {i+1} - C{c}')

plt.tight_layout()
plt.show()


In [ ]:
print([int(i) for i in inputs['input_ids'][0]])

In [ ]:
text

In [ ]:
"""
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
<|vision_start|><|video_pad|><|vision_end|><|vision_start|><|video_pad|><|vision_end|>这段校园步行道视频里有异常情况吗？<|im_end|>
<|im_start|>assistant

"""